In [1]:
from flask import Flask, request, jsonify
import joblib
import pandas as pd
# Load the saved random forest model for texture: wood
loaded_model = joblib.load('wood_RF_model.joblib')

#collison flag
collisionOnflag = False

app = Flask(__name__)

@app.route('/collisonRegister', methods=['POST'])
def collisonRegister():
    global collisionOnflag
    try:
        collisionStatus = request.json.get('CollisionStatus')  
        textureName = request.json.get('TextureName')
        
        if collisionStatus is None:
            return jsonify({'Status': '400', 'Error': 'No data received'})
        
        elif collisionStatus not in ['ON', 'OFF']:
            return jsonify({'Status': '400', 'Error': 'Invalid CollisionStatus'})
        
        elif collisionStatus == "ON" and collisionOnflag == False:
            if textureName is None or textureName.strip() == '':
                return jsonify({'Status': '400', 'Error': 'Texture name is empty or null'})
            else:
                #start to listen input for the cane velocity
                #for each velocity input from the cane, use the current texture in use to call the prediction model
                #return the predicted DFT321 waveform value
                collisionOnflag = True
                return jsonify({'Status': '200', 'Message': 'CollisionStatus is ON'})
        elif collisionStatus == "OFF" and collisionOnflag == True:  
            #collisionStatus is 'OFF'
            #stop listening to the cane input
            collisionOnflag = False
            return jsonify({'Status': '200', 'Message': 'CollisionStatus is OFF'})
        else:
            return jsonify({'Status': '400', 'Message': 'CollisionStatus is ALREADY ON'})
    
    except Exception as e:
        return jsonify({'Status': '500', 'Error': str(e)} )
    
@app.route('/predict', methods=['POST'])
def predict():
    global collisionOnflag
    if collisionOnflag == True:
        try:
                data = request.get_json()
                # Convert JSON data to a Python dictionary
                vector_data = {
                    'VELO-X': data.get('x'),
                    'VELO-Y': data.get('y'),
                    'VELO-Z': data.get('z')
                }

                x_input_df = pd.DataFrame([vector_data])
                y_pred = loaded_model.predict(x_input_df)

                return jsonify({'Status': '200', 'Message': str(y_pred)})
        except Exception as e:
                return jsonify({'Status': '500', 'Error': str(e)}) 
    else:
        return jsonify({'Status': '400', 'Error': 'Collision flag is not set to true'}) 



if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.4.57:5000
Press CTRL+C to quit
127.0.0.1 - - [30/Dec/2023 16:48:48] "POST /collisonRegister HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2023 16:48:48] "POST /collisonRegister HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2023 16:48:48] "POST /collisonRegister HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2023 16:48:48] "POST /collisonRegister HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2023 16:48:49] "POST /collisonRegister HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2023 16:48:49] "POST /collisonRegister HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2023 16:48:54] "POST /collisonRegister HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2023 16:48:54] "POST /collisonRegister HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2023 16:48:54] "POST /collisonRegister HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2023 16:48:54] "POST /collisonRegister HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2023 16:48:54] "POST /collisonRegister HTTP/1.1" 200 -
127.0.0.1 - - [30/D